# 基于高斯混合变分自编码器的深度无监督聚类  
  
## 常规VAE中存在的问题  
  
* 1、过度正则化项问题    
  
  V-AE文献中多次描述了正则化项对V-AE训练的过度影响。由于受先验信息的强烈影响，所得到的潜在表示往往过于简化，不能很好地反映数据的底层结构。 到目前为止，有两种主要的方法来克服这种影响：一种解决方案是在训练期间退火KL项，允许重建项训练自动编码器网络，然后缓慢地纳入KL项的正则化。 另一种主要方法是通过设置一个临界值来修改目标函数，当KL项低于某个阈值时，该值会消除KL项的影响  
  
  <br>
  
* 2、噪声变量z,过于简单不能表达复杂的潜在变量结构。   
      
   isotropic：各方向同性。（性质不会因为方向的变化而出现改变。）  
   在常规的变分自编码器中，潜在变量的先验分布通常是同性的高斯分布。选择这种先验分布会造成（样本的）多元高斯分布的每一维，都是从数据中学得的一个独立的连续变量因子。这就造成最终的生成样本的特征是结构化的，相互分离的  
   常规VAE,的噪声变量z都是从同一个高斯分布中抽样的，（各方向同性），（真实样本是服从多维正态分布的）导致最终生成样本的每一维度都是相互独立，相互分离的。（生成样本模糊)  
   虽然可解释性更好，但是每个特征都是单峰的，不能表示更复杂的特征。  
   
  <br>  
  
* 3、无法完成聚类任务。  
  
<br>    

## GMVAE(聚类模型)    
  
  GMVAE聚类模型，这个模型的噪声变量z的抽样不再是来自于一个标准正态分布N(0,1),而是来自一个高斯混合模型GMM。GMVAE还同时解决了过度正则化项问题（over-regularisation）
  
#### 生成模型（相当于VAE的解码器）   
  
 GMVAE的生成结构的设计：想法是从高斯混合模型中抽样，但是同时又要保留分布的参数以便于BP（求梯度）。所以用重参数技巧来设计这个结构。这个结构的设计为：1、从标准正态分布W中抽样。2、从k个类被的均匀分布中抽样选择一个类别形成one-hot向量z 3、用 w,z经过神经网络$\beta$生成类别z对应的多维高斯正态分布x这个潜在变量。 4、从 多维高斯正态分布x这个潜在变量中抽样潜在变量,经过神经网络$\theta$生成样本y。
  
 生成模型的结构：  
![](img/1.PNG)  
在这个生成模型里有：{y},{wz}条件独立于x，w与z相互独立。所以生成模型的概率公式为：  
$P_{\beta,\theta}(y,x,w,z)=P_{\beta,\theta}(y,w,z|x)P(x)=P_{\theta}(y|x)P_{\beta}(w,z|x)P(x)=P_{\theta}(y|x)P_{\beta}(x,w,z)=P_{\theta}(y|x)P_{\beta}(x|w,z)P(w,z)=P_{\theta}(y|x)P_{\beta}(x|w,z)P(w)P(z)$  
  
最终可得：  
$P_{\beta,\theta}(y,x,w,z)=P_{\theta}(y|x)P_{\beta}(x|w,z)P(w)P(z)$  

<br>
  
 其中设定变量：  
    
  $W\text{~}N(0,1) \qquad\qquad$为重参数技巧中得标准正态分布N(0,1)  
    
  $Z\text{~}Mult(\pi) \qquad\qquad$为多项分布，其中K为类别数，$\pi_k=K^{-1}$,即多项分布为选择每个类别的概率相同（均匀分布）  
    
  $x|w  \qquad\qquad\qquad$为通过参数为$\beta$的神经网络，将输入的w产生，K个均值向量和K个方差对角阵，即产生了K个多维正态分布。组成高斯混合模型，  
  $\qquad\qquad\qquad\qquad$只是没有每个组件对应的系数  
    
  $x|w,z \text{~} \prod_{k=1}^K \; N(u_{zk}(w;\beta),diag(\sigma^2_{zk}(w;\beta)\;)\;)^{zk} \qquad$每个k对应一个多维正态分布。这个决定了x是自哪个模态，（zk取值是0or1）（即来自哪个组件）  
    
  $y|x \text{~} \; N(u(x;\theta),diag(\sigma^2(x;\theta)\;)\;) \qquad $通过参数为$\theta$的神经网络,将从某个潜在模态中采样的潜在变量x，生成样本y. （真实样本y服从的也是多维正态分布。）    
    
  <br>  
  
  $N(0,1)\rightarrow N(U,\sum) \text{多维正态分布的潜在抽样变量} \rightarrow N(U,\sum)\text{样本服从的多维正态分布}$  
    
<br>   
     
#### 识别模型（相当于VAE的编码器）   
  
  按上述描述进行变分自编码器的设计之后，优化过程的需要使用变分推断，所以现在我们为变分推断，假设一个简单的带参数分布，也就是平均场变分族：  
  ![](img/2.PNG)  
   
  简单带参变分族为：$q_{\phi x}(x|y)q_{\phi w}(w|y)p_{\beta}(z|x,w)$   
    
   通过调整这个变分族，让它靠近真实后验分布$q_{true}(x,w,z|y)$  
   
  
  则有下界(ELBO)公式为：  
    
  $$log p(y)=E_{q_{mvf}\;(x,w,z|y)\;}[log \frac{P_{\beta,\theta}(y,x,w,z)}{q_{mvf}\;(x,w,z|y)}] - KL(q_{mvf}\;(x,w,z|y)||q_{true}(x,w,z|y))$$    
    
  其中 ：  
  $$q_{mvf}\;(x,w,z|y)=q_{\phi x}(x|y)q_{\phi w}(w|y)p_{\beta}(z|x,w) \quad\text{变分族}$$   
    
  $$L_{ELBO}=E_{q_{mvf}\;(x,w,z|y)\;}[log \frac{P_{\beta,\theta}(y,x,w,z)}{q_{mvf}\;(x,w,z|y)}]$$ 
    
  
   
变分族中的$p_{\beta}(z|x,w)$可以具体的推导为（这里x每一次为一个样本）：  
  
  $$p_{\beta}(z_j =1|x,w)=\frac{P(z_j=1)P(x|z_j=1,w)}{\sum_{k=1}^K P(z_k=1)P(x|z_k=1,w)}$$  
    
  $$\qquad\qquad\qquad= \frac{\pi_j N(x|u_j(w;\beta),\sigma_j(w;\beta))}{\sum_{k=1}^K \pi_k N(x|u_k(w;\beta),\sigma_k(w;\beta)}$$  
    
<br>  

所以具体的变分下界为（ELBO）:   
  
  $L_{ELBO}=E_{q_{mvf}\;(x,w,z|y)\;}[log \frac{P_{\beta,\theta}(y,x,w,z)}{q_{mvf}\;(x,w,z|y)}]$ 
  
  $\qquad\;\;=E_{q_{\phi x}(x|y)q_{\phi w}(w|y)p_{\beta}(z|x,w)\;}[log \frac{P_{\theta}(y|x)P_{\beta}(x|w,z)P(w)P(z)}{q_{\phi x}(x|y)q_{\phi w}(w|y)p_{\beta}(z|x,w)}] $  
    
  $\qquad\;\;=E_{q_{\phi x}(x|y)q_{\phi w}(w|y)p_{\beta}(z|x,w)\;}[log P_{\theta}(y|x) + log P_{\beta}(x|w,z)+log P(w)+log P(z)-log q_{\phi x}(x|y)-log q_{\phi w}(w|y)- logp_{\beta}(z|x,w)]$   
    
   $\qquad\;\;=E_{q_{\phi x}(x|y)q_{\phi w}\;(w|y)p_{\beta}(z|x,w)\;}[log P_{\theta}(y|x) -log \frac{q_{\phi x}\;(x|y)}{P_{\beta}(x|w,z)}-log\frac{q_{\phi w}\;(w|y)}{ P(w)}-log \frac{p_{\beta}(z|x,w)}{P(z)}]$  
     
   $\quad\;\;=E_{q_{\phi x}(x|y)\;}[log P_{\theta}(y|x)] -E_{q_{\phi w}\;(w|y)p_{\beta}(z|x,w)\;}[KL(q_{\phi x}\;(x|y)||P_{\beta}(x|w,z)]-KL(q_{\phi w}\;(w|y)|| P(w))-E_{q_{\phi x}(x|y)q_{\phi w}\;(w|y)\;}[KL(p_{\beta}(z|x,w) || P(z))]$  
     
 这些就是最终的变分下界（ELBO）   
   
 各项分别为：重构项、条件先验项、w-先验项、z-先验项   
   
<br>  

（根据VAE的推导，可以知道我们这个下界是没有问题，z-先验项也没有问题。但是这个z-先验项，只是用来保持样本的类别的编码信息。
不能让它过度正则化，过度正则化，达到完美的0，达到完美的反聚类效果（类别完全重合），聚类性能反而变差了。）  
  
<br>  

* 条件先验项  
  
 在上述的下界ELBO中： 重建项可以通过从q(x|y)提取蒙特卡罗样本来估计，其中梯度可以用标准的重参数化技巧反向传播。w-先验项可以解析计算。  
  **而条件先验项的计算方式为：**    
$$E_{q_{\phi w}\;(w|y)p_{\beta}(z|x,w)\;}[KL(q_{\phi x}\;(x|y)||P_{\beta}(x|w,z)] \approx \frac{1}{M}\sum_{j=1}^M \sum_{k=1}^K p_{\beta}(z_k=1|x^j,w^j) KL(q_{\phi x}\;(x|y)||P_{\beta}(x|w^j,z_k=1)$$     
即 $q_{\phi w}\;(w|y)$使用蒙特卡洛采样，其梯度可以用标准的重参数化技巧反向传播。而$p_{\beta}(z|x,w)$可以通过计算有z指，来计算其期望，并使用标准BP计算梯度。  

<br>  
  
<br>  

## GMVAE 存在的问题  
  
**过度正则化问题**   （分析z-先验项所造成的影响：离散潜变量Z的KL代价：过度正则化问题）  
  
 $-E_{q_{\phi x}(x|y)q_{\phi w}\;(w|y)\;}[KL(p_{\beta}(z|x,w) || P(z))]$  
   
$\color{red}{\text{ 用来衡量不同集群重叠程度的度量。（即一个样本如果来自每个类别的概率相同，说明所有类别完全重叠。}}$     
$\color{red}{\text{最理想的情况来自一个类别的开率为1，来自其他类别的概率为0。）。所以这个z-先验项越小，说明重叠程度越小，越接近0说明重叠程度越大。}}$     
  
**也就是说这个z-先验项具有反聚类效果**    
  
我们认为这种现象可以解释为z-先验项过度正则化的结果。考虑到这个数量是由下界中KL项的优化所驱动的，它达到了其最大可能值0，而不是通过训练来减少，以确保关于类的信息的编码。  
  
**这个现象也就是GMVAE中的过度正则化问题**    
  
<br>  

**解决方案：**  
  
  **通过使用前面提到的对Kingma等人提出的下界的修改。（2016年），我们可以避免z-先验导致的过度正则化。这是通过将z先验的代价保持在一个常量值λ，直到它超过该阈值。形式上，修改后的z-先验项写为**  
    
 $$ = -max(\lambda, E_{q_{\phi x}(x|y)q_{\phi w}\;(w|y)\;}[KL(p_{\beta}(z|x,w) || P(z))])$$ 
    
  **这种修改抑制了z-prior合并所有簇的初始效果，从而允许它们分散开来，直到z-prior代价足够高。在这一点上，它的影响被大大降低，并且主要局限于合并足够重叠的单个集群。**
    
      
<br>  
  
**所以最终的目标函数就是求这个变分下界：**  
$L_{ELBO}=E_{q_{\phi x}(x|y)\;}[log P_{\theta}(y|x)] -E_{q_{\phi w}\;(w|y)p_{\beta}(z|x,w)\;}[KL(q_{\phi x}\;(x|y)||P_{\beta}(x|w,z)]-KL(q_{\phi w}\;(w|y)|| P(w)) $  
  
 $\qquad\qquad -max(\lambda, E_{q_{\phi x}(x|y)q_{\phi w}\;(w|y)\;}[KL(p_{\beta}(z|x,w) || P(z))])$   
    
      
<br>  
  
<br>   

<br>  
  
#### 个别易混淆问题  
  
  1、在生成模型中，我们选择的z是多项均匀分布。（也就是我们均匀的选择一个类别，去生成这个类别的样本y）  
    
  2、在推断模型中。我们推断一个y是属于哪个类别的（KL(P(z|w,x)||p(z))）,**这里乍一看我们好像是尽可能尽可能让一个样本属于每一个类别的概率都相同（这就是GMVAE的过正则化问题）**   
    
$\color{red}{\text{z在生成模型和推断模型中的意义是不同的。在生成模型中是均匀选择一个类别生成样本。在推断模型中只是保留一个样本y的类别编码(不能过正则化)}}$